In [ ]:
from IPython.display import Image
from PIL import Image
from struct import unpack
import os
from tqdm import tqdm

Extract images and move them into a single folder

In [ ]:
!unzip /content/drive/MyDrive/AIRecognition.zip -d AIRecognition
!unzip /content/drive/MyDrive/Datasets/art_images.zip -d art_images
!unzip /content/drive/MyDrive/Datasets/Abstract_gallery.zip -d Abstract_gallery
! mv /content/Abstract_gallery /content/AIRecognition/real/Abstract_gallery
!mv /content/art_images/art_images /content/AIRecognition/real/art_images

Delete files with invalid extensions and rename JPEG to JPG

In [ ]:
directory_path = '/content/AIRecognition/real'

files = os.listdir(directory_path)
file_extensions = [os.path.splitext(file)[1] for file in files if os.path.isfile(os.path.join(directory_path, file))]
unique_extensions = set(file_extensions)
print(unique_extensions)

! rm /content/AIRecognition/real/*.txt
! rm /content/AIRecognition/fakeV2/fake-v2/sort
! rm /content/AIRecognition/fakeV2/fake-v2/dataset-metadata.json

files = os.listdir(directory_path)
for file in files:
    file_path = os.path.join(directory_path, file)
    if os.path.isfile(file_path) and file.lower().endswith('.jpeg'):
        new_file_path = os.path.join(directory_path, file[:-5] + '.jpg')
        os.rename(file_path, new_file_path)
        print(f"Renamed {file_path} to {new_file_path}")

Remove corrupted images using PIL Image library

In [ ]:
Image.MAX_IMAGE_PIXELS = None

def is_image_corrupted(image_path):
    try:
        img = Image.open(image_path)
        img.verify()  # Verify that it is an image
        return False
    except (IOError, SyntaxError) as e:
        print(f'Corrupted image: {image_path}')
        return True

directory = '/content/AIRecognition/real/'  # done for /fake directory also
for root, dirs, files in os.walk(directory):
        for file in files:
            image_path = os.path.join(root, file)
            if is_image_corrupted(image_path):
              os.remove(image_path)

In [ ]:
Remove images that canno tbe decoded using tensorflow image library

In [ ]:
directory = '/content/AIRecognition/real/'
def preprocess_image(file_path):
    image = tf.io.read_file(file_path)
    try:
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, [img_height, img_width])
    except tf.errors.InvalidArgumentError:
        print(f"Error decoding image: {file_path}")
        return None
    return image

for root, dirs, files in os.walk(directory):
        for file in files:
            image_path = os.path.join(root, file)
            preprocess_image(image_path)

Additional check for corrupted images

In [ ]:
from struct import unpack
import os
from tqdm import tqdm

marker_mapping = {
    0xffd8: "Start of Image",
    0xffe0: "Application Default Header",
    0xffdb: "Quantization Table",
    0xffc0: "Start of Frame",
    0xffc4: "Define Huffman Table",
    0xffda: "Start of Scan",
    0xffd9: "End of Image"
}


class JPEG:
  def __init__(self, image_file):
    with open(image_file, 'rb') as f:
        self.img_data = f.read()

  def decode(self):
    data = self.img_data
    while(True):
        marker, = unpack(">H", data[0:2])
        if marker == 0xffd8:
            data = data[2:]
        elif marker == 0xffd9:
            return
        elif marker == 0xffda:
            data = data[-2:]
        else:
            lenchunk, = unpack(">H", data[2:4])
            data = data[2+lenchunk:]
        if len(data)==0:
            raise TypeError("issue reading jpeg file")


bads = []

img_dir = '/content/AIRecognition/real/'
for dirName, subdirList, fileList in os.walk(img_dir):
  imagesList = fileList
  for img in tqdm(imagesList):
    if img.lower().endswith('.jpg'):
      image = JPEG(img_dir + img)
      try:
        image.decode()
      except:
        bads.append(img)


print(bads)
for name in bads:
  os.remove(os.path.join(img_dir, name))